# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [17]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.pipeline.core import Pipeline

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-experiment'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [12]:
project_name = 'azureml-spaceship-titanic'
project_folder = os.getcwd().split(project_name)[0] + project_name

In [13]:
import sys
sys.path.append(project_folder)

from src.pipelines.preprocess import preprocess_data

df = pd.read_csv('../data/01_raw/train.csv')
processed_df = preprocess_data(df)#.drop(columns=["PassengerId"])


In [10]:
found = False
dataset_name = "Spaceship_Dataset"
description_text = "Data to predict which passengers are transported to an alternate dimension"


if dataset_name in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[dataset_name]

if not found:
        # Create AML Dataset and register it into Workspace
        dataset = Dataset.Tabular.register_pandas_dataframe(processed_df, ws.get_default_datastore(), dataset_name)   
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=dataset_name,
                                   description=description_text,
                                   create_new_version=True
                                )


df = dataset.to_pandas_dataframe()
df.describe()

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,People_in_Cabin_Num,People_in_Cabin_Deck,Family_Size,Group_Size
count,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000,8494.000000,8494.000000,8493.000000,8693.000000
mean,224.687617,458.077203,173.729169,311.138778,304.854791,8.037203,1952.105957,5.428117,2.035546
std,666.717663,1611.489240,604.696458,1136.705535,1145.717189,5.214669,963.741893,2.891901,1.596347
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,779.000000,3.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,2559.000000,5.000000,1.000000
75%,47.000000,76.000000,27.000000,59.000000,46.000000,12.000000,2794.000000,7.000000,3.000000
max,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000,28.000000,2794.000000,18.000000,8.000000


In [11]:
df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Transported', 'Cabin_Deck', 'Cabin_Side', 'Cabin_Region',
       'People_in_Cabin_Num', 'People_in_Cabin_Deck', 'Family_Size',
       'Group_Size', 'Age_Cat'],
      dtype='object')

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [14]:
project_folder

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook250837/code/Users/odl_user_250837/azureml-spaceship-titanic'

In [15]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Transported",   
                             path = project_folder + '/automl-pipeline',
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [18]:
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.widgets import RunDetails

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)


pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

pipeline_run = experiment.submit(pipeline)
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()


Created step automl_module [e71f3467][731b4bfd-b0ff-4c37-b289-9cd0a4927717], (This step will run and generate new outputs)
Submitted PipelineRun 7693fd7d-5ea0-4d7a-89b1-25400fb85c07
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/7693fd7d-5ea0-4d7a-89b1-25400fb85c07?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-250837/workspaces/quick-starts-ws-250837&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: 7693fd7d-5ea0-4d7a-89b1-25400fb85c07
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/7693fd7d-5ea0-4d7a-89b1-25400fb85c07?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-250837/workspaces/quick-starts-ws-250837&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: Running


StepRunId: 2515ed9f-6fdd-4dfa-a546-ed5af532b702
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/2515ed9f-6fdd-4dfa-a546-ed5af532b702?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-250837/workspaces/quick-starts-ws-250837&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/py

'Finished'

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [19]:
import json

metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

Downloaded azureml/2515ed9f-6fdd-4dfa-a546-ed5af532b702/metrics_data, 1 files out of an estimated total of 1


,2515ed9f-6fdd-4dfa-a546-ed5af532b702_1,2515ed9f-6fdd-4dfa-a546-ed5af532b702_3,2515ed9f-6fdd-4dfa-a546-ed5af532b702_4,2515ed9f-6fdd-4dfa-a546-ed5af532b702_39,2515ed9f-6fdd-4dfa-a546-ed5af532b702_26,2515ed9f-6fdd-4dfa-a546-ed5af532b702_29,2515ed9f-6fdd-4dfa-a546-ed5af532b702_31,2515ed9f-6fdd-4dfa-a546-ed5af532b702_32,2515ed9f-6fdd-4dfa-a546-ed5af532b702_30,2515ed9f-6fdd-4dfa-a546-ed5af532b702_0,...,2515ed9f-6fdd-4dfa-a546-ed5af532b702_10,2515ed9f-6fdd-4dfa-a546-ed5af532b702_21,2515ed9f-6fdd-4dfa-a546-ed5af532b702_35,2515ed9f-6fdd-4dfa-a546-ed5af532b702_55,2515ed9f-6fdd-4dfa-a546-ed5af532b702_47,2515ed9f-6fdd-4dfa-a546-ed5af532b702_48,2515ed9f-6fdd-4dfa-a546-ed5af532b702_45,2515ed9f-6fdd-4dfa-a546-ed5af532b702_49,2515ed9f-6fdd-4dfa-a546-ed5af532b702_54,2515ed9f-6fdd-4dfa-a546-ed5af532b702_46
recall_score_macro,[0.7972977455515071],[0.7541344150436271],[0.7865222183552779],[0.7871665856332054],[0.7933871240694247],[0.7952283336555069],[0.8004115688910537],[0.7951648591884944],[0.7947543767542355],[0.8009402983881991],...,[0.7875852661002153],[0.7905179938526858],[0.8110092547067413],[0.8113128471313807],[0.7913898954592246],[0.8025400768322951],[0.7668496599010597],[0.7730037022747943],[0.8097289184566027],[0.7887905376094423]
precision_score_macro,[0.797459986365165],[0.7608797893022977],[0.7868094967967295],[0.788783627381286],[0.7936100820469839],[0.7955551390172334],[0.8004273293280432],[0.7953072735904883],[0.7949253278620652],[0.8010361792598967],...,[0.7883796742418658],[0.7905625185600232],[0.8110324682912845],[0.8117904897054297],[0.7915854011634137],[0.8025833239487579],[0.7704949063106353],[0.7733515997577348],[0.809838738190809],[0.7889048698030271]
AUC_weighted,[0.8901487826031319],[0.8462652394431619],[0.8667697455512835],[0.8609604000389166],[0.8780861457554576],[0.879007566391636],[0.8889028874378818],[0.8832858446985662],[0.8811952244932245],[0.8952172122911907],...,[0.8690534000172933],[0.8840394530051844],[0.8994562617111708],[0.901978172903067],[0.8829397468642665],[0.8848708298461392],[0.8516857725851209],[0.8506387013595976],[0.9027071442209623],[0.8800504628271925]
precision_score_micro,[0.7971934032326343],[0.7537086309429513],[0.7863794828646024],[0.7874146001444106],[0.7933958556716733],[0.7952369597099528],[0.8004140151488984],[0.7952375155628896],[0.7947768325895623],[0.8007590807113552],...,[0.7876447629640587],[0.7904057639090882],[0.8109972168443451],[0.8112273002564309],[0.7913260578536502],[0.8024844482274207],[0.766248752606454],[0.7726900955503258],[0.8097317382259827],[0.7885655730577764]
f1_score_weighted,[0.7971657391441532],[0.7521029574800612],[0.7863236636303977],[0.7871232708777268],[0.7933611826704814],[0.7951833315078066],[0.8004125151698788],[0.7952137321043583],[0.7947505661352929],[0.8007427466996041],...,[0.7875037491611673],[0.7903983327115185],[0.8109946385964318],[0.8111579876233708],[0.7912925954690149],[0.8024777701969796],[0.7653995072677352],[0.7726153427146926],[0.809716536253561],[0.7885446395542287]
accuracy,[0.7971934032326343],[0.7537086309429513],[0.7863794828646024],[0.7874146001444106],[0.7933958556716733],[0.7952369597099528],[0.8004140151488984],[0.7952375155628896],[0.7947768325895623],[0.8007590807113552],...,[0.7876447629640587],[0.7904057639090882],[0.8109972168443451],[0.8112273002564309],[0.7913260578536502],[0.8024844482274207],[0.766248752606454],[0.7726900955503258],[0.8097317382259827],[0.7885655730577764]
norm_macro_recall,[0.5945954911030144],[0.5082688300872542],[0.5730444367105558],[0.5743331712664107],[0.5867742481388496],[0.5904566673110136],[0.6008231377821073],[0.590329718376989],[0.589508753508471],[0.6018805967763982],...,[0.5751705322004307],[0.5810359877053716],[0.6220185094134826],[0.6226256942627616],[0.582779790918449],[0.6050801536645903],[0.5336993198021193],[0.5460074045495885],[0.6194578369132052],[0.5775810752188845]
matthews_correlation,[0.5947576880621953],[0.5149618490021414],[0.5733315928231887],[0.5759465795661782],[0.5869969937966

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [20]:
import pickle

# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

Downloaded azureml/2515ed9f-6fdd-4dfa-a546-ed5af532b702/model_data, 1 files out of an estimated total of 1


PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mn...
                                                  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('35', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l1')), ('lightgbmclassifier', LightGBMClassifier(boosting_type='gbdt', colsample_bytree=0.3966666666666666, learning_rate=0.06842421052631578, max_bin=360, max_depth=2, min_child_weight=8, min_data_in_leaf=1e-05, min_split_gain=0.9473684210526315, n_estimators=600, n_jobs=1, num_leaves=200, problem_info=ProblemInfo(gpu_training

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

In [40]:
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = 'http://5708e1ea-8382-4a27-ad5b-1b69c6f1577c.westus2.azurecontainer.io/score'
# If the service is authenticated, set the key or token
key = 'acrRxUPvQ3c5u5FmyqngHDbkMGNKrczT'

# Two sets of data to score, so we get two results back
data = {
    "Inputs": {
        "data":
            [
              {
        "PassengerId": "0001_01",
        "HomePlanet": "Europa",
        "CryoSleep": "False",
        "Destination": "TRAPPIST-1e",
        "VIP": "False",
        "RoomService": 109.00,
        "FoodCourt": 1000,
        "ShoppingMall": 25.0,
        "Spa": 200.0,
        "VRDeck": 2.0,
        "Cabin_Deck": "B",
        "Cabin_Side": "P",
        "Cabin_Region": "A",
        "People_in_Cabin_Num": 14,
        "People_in_Cabin_Deck": 700,
        "Family_Size": 4,
        "Group_Size": 2,
        "Age_Cat": "Pre_Adult"
        }
      ]
    },
    "GlobalParameters": {
        "method": "predict"
    }
}
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())


{'Results': [True]}


TODO: In the cell below, print the logs of the web service and delete the service

In [49]:
%%writefile logs.py
from azureml.core import Workspace, Webservice
import time

# Load the Azure ML workspace
ws = Workspace.from_config()

# Name of the web service
service_name = 'web-service-automl'

# Get a reference to the web service
service = Webservice(workspace=ws, name=service_name)

# Print logs
print("Web service logs:")
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

service.wait_for_deployment()

# Delete the deployed web service
service.delete()
print("Web service deleted successfully.")

Overwriting logs.py


In [50]:
!python logs.py

Web service logs:
2024-01-28T01:43:12,289393980+00:00 - gunicorn/run 
2024-01-28T01:43:12,283433980+00:00 - rsyslog/run 
2024-01-28T01:43:12,292718080+00:00 | gunicorn/run | 
2024-01-28T01:43:12,295909680+00:00 | gunicorn/run | ###############################################
2024-01-28T01:43:12,298100279+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-01-28T01:43:12,300920679+00:00 | gunicorn/run | ###############################################
2024-01-28T01:43:12,305886479+00:00 | gunicorn/run | 
2024-01-28T01:43:12,321552579+00:00 | gunicorn/run | 
2024-01-28T01:43:12,337595479+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20231216.v3
2024-01-28T01:43:12,346705579+00:00 - nginx/run 
2024-01-28T01:43:12,348981879+00:00 | gunicorn/run | 
2024-01-28T01:43:12,356810279+00:00 | gunicorn/run | 
2024-01-28T01:43:12,360197179+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
